### First approach to prefix autocomplete problem

In [1]:
import pandas as pd

In [33]:
raw_data = pd.read_csv('data/train.csv', 
                        encoding='ISO-8859-1',
                        usecols=[1, 3, 4],
                        index_col=[1])
raw_data.sample(5, random_state=0)

,product_uid,relevance
search_term,,
outdoor stair tread,113211,2.33
steele stake,184373,3.00
margarita,107103,2.67
sheet metal,103441,3.00
top bar shower door,185541,2.33


In [34]:
lex_sorted_data = raw_data.sort_index()
lex_sorted_data.reset_index(inplace=True)
lex_sorted_data.set_index(['search_term', 'product_uid'], inplace=True)
lex_sorted_data

relevance
search_term  product_uid           
$ hole saw   100490            3.00
             165925            3.00
             125403            2.33
             121588            2.33
             197110            1.33
...                             ...
zwave switch 142033            3.00
             141628            3.00
             134888            2.00
             113534            2.33
             152640            2.67

[74067 rows x 1 columns]

In [28]:
promt = 'sin'
index_for_bin_search = lex_sorted_data.index.get_level_values('search_term')
n = index_for_bin_search.searchsorted(promt)
idx = lex_sorted_data.iloc[n:].groupby('search_term', sort=False).max()[:5]
idx.sort_values('relevance', ascending=False)

,relevance
search_term,
sing,3.0
singel wall stove pipe,3.0
single 14 wide shelf,3.0
single angle sanding block,3.0
single board pannel,2.0


In [18]:
lex_sorted_data.loc[idx.index]

,product_uid,relevance
search_term,,
sing,151187,1.00
sing,163459,1.67
sing,140305,1.00
sing,133967,3.00
singel wall stove pipe,125523,3.00
singel wall stove pipe,148616,2.00
singel wall stove pipe,184297,3.00
singel wall stove pipe,154581,2.67
singel wall stove pipe,179142,2.33


In [19]:
brands = pd.read_csv('data/attributes.csv',
                        index_col=[1, 0],
                        dtype={'product_uid': 'Int64'}).loc['MFG Brand Name']

In [35]:
list(filter( lambda x: x in brands.index, lex_sorted_data.loc[idx.index].nlargest(5, 'relevance').index.get_level_values('product_uid')))

[125523, 184297, 197173]

In [8]:
brands.loc[filter( lambda x: x in brands.index, lex_sorted_data.loc[idx.index].nlargest(5, 'relevance').index.get_level_values('product_uid'))]['value'].to_list()#.groupby('product_uid', sort=True).max()

['DuraVent', 'Speedi-Products', 'Sandusky']

In [116]:
lex_sorted_data.loc[idx.index].sort_values('relevance', ascending=False)[:5]

relevance
search_term   product_uid           
test leads    114045            3.00
texture tools 149558            3.00
              111958            3.00
test leads    112860            3.00
texture gun   114227            2.67

In [10]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [11]:
stemmer = PorterStemmer()

In [12]:
stemmer.stem('direction')

'direct'

In [13]:
[stemmer.stem(word) for word in word_tokenize('This is me, Jimbo')]

128 µs ± 5.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/arthur/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Train data preprocessing

In [63]:
import requests

In [65]:
def speller(s: str) -> str:
    res = s
    try:
        response = requests.get(f'https://speller.yandex.net/services/spellservice.json/checkText?text={s}').json()
    except requests.exceptions.ConnectionError:
        response.status_code = "Connection refused"

    for j in response:
        res = s.replace(j['word'], j['s'][0])
    return res

In [66]:
import tqdm
from time import sleep

spell_corrected_data = lex_sorted_data.copy()

In [68]:
for term in tqdm.tqdm(lex_sorted_data.index.get_level_values('search_term').unique()):
    correct = speller(term)
    if correct != term:
        spell_corrected_data.rename(index={term: correct}, inplace=True)

100%|██████████| 11795/11795 [31:28<00:00,  6.24it/s]


In [69]:
#spell_corrected_data.to_csv('preprocessed.csv')

In [7]:
import pandas as pd
raw_data = pd.read_csv('data/preprocessed.csv',
                        index_col=[0, 1])
raw_data.head(50)#, random_state=0)

relevance
search_term               product_uid           
$ hole saw                100490            3.00
                          100712            3.00
                          111472            2.00
                          121588            2.33
                          125403            2.33
                          133315            2.00
                          165925            3.00
                          197110            1.33
. exterior floor stain    102253            3.00
                          102836            2.33
                          111283            2.67
.105 trimmer line         120307            3.00
                          131403            3.00
                          142574            2.33
.110 wire connector       103075            2.33
                          103297            2.00
                          127687            2.00
                          169775            2.00
                          171077            2.00
                          172086            2.33
                          173029            2.67
                          174323            1.67
                          178918            2.00
.440 metal screws         107178            1.33
                          109031            2.33
                          133273            2.67
                          142783            2.33
                          143108            2.33
                          152423            3.00
                          154351            2.00
                          161398            2.33
.5' brad nails            133249            1.67
                          137444            2.33
                          152672            2.67
                          153135            2.67
                          158820            2.67
                          162809            2.67
.75 in pocket hole screws 108020            2.00
                          121548            2.00
                          145024            2.67
                          178082            3.00
                          188015            3.00
.75 x 3                   175012            1.00
                          201555            2.33
.8 mulch rubber vigorous  101975            3.00
                          123218            3.00
.875 arbor wire wheel     104044            2.67
                          139275            2.00
                          160103            2.33
                          164788            2.67

In [128]:
brands = pd.read_csv('data/attributes.csv',
                        index_col=[1, 0],
                        dtype={'product_uid': 'Int64'}).loc['MFG Brand Name']

In [131]:
brands.loc[112860]

KeyError: 112860